# torchrec MovieLens Tutorial

## Table of contents
1. Instantiating MovieLens-25M dataset
2. Defining model
3. Training and evaluating model
4. Finding similar movies

## 1. Instantiating MovieLens-25M dataset

To start, we can load the MovieLens-25M dataset using `torchrec.datasets.movielens.movielens_25m`. The function loads just the user-movie ratings data in `ratings.csv` by default; we call the function with `include_movies_data=True` such that it adds movie data from `movies.csv` to each user-movie sample.

In [1]:
from torchrec.datasets.movielens import movielens_25m

dp = movielens_25m("/home/jeffhwang/local/datasets/ml-25m", include_movies_data=True)

Let's check out a single sample.

In [2]:
next(iter(dp))

{'userId': 1,
 'movieId': 296,
 'rating': 5.0,
 'timestamp': 1147880044,
 'title': 'Pulp Fiction (1994)',
 'genres': 'Comedy|Crime|Drama|Thriller'}

Seems reasonable.

Next, we instantiate datapipes representing training and validation data splits and apply shuffling and batching.

In [5]:
from torchrec.datasets.utils import rand_split_train_val

train_dp, val_dp = rand_split_train_val(dp, 0.9)
batched_train_dp = train_dp.shuffle(buffer_size=int(1e5)).batch(8192)
batched_val_dp = val_dp.batch(8192)

Turns out that the integer user ids and movie ids referenced by the dataset aren't contiguous. Let's remap them to contiguous values so that we can use them with `torch.nn.Embedding` more easily downstream.

To do so, we first populate dictionaries that map movie and user ids to ids in contiguous ranges

In [6]:
contig_movie_ids = {}
contig_user_ids = {}
movie_id_to_title_genre = {}

available_movie_id = 0
available_user_id = 0
for sample in dp:
    if sample["movieId"] not in contig_movie_ids:
        contig_movie_ids[sample["movieId"]] = available_movie_id
        available_movie_id += 1
    if sample["userId"] not in contig_user_ids:
        contig_user_ids[sample["userId"]] = available_user_id
        available_user_id += 1
    movie_id_to_title_genre[sample["movieId"]] = (sample["title"], sample["genres"])

, and then define a function `_transform` that uses those dictionaries to remap movie and user ids for a batch of data. While we're at it, we'll also have `_transform` reformat the batch as tensors representing user ids, movie ids, and labels (numerical movie ratings given by users).

In [7]:
import torch

def _transform(batch):
    user_ids = torch.tensor([contig_user_ids[sample["userId"]] for sample in batch], dtype=torch.int32)
    movie_ids = torch.tensor([contig_movie_ids[sample["movieId"]] for sample in batch], dtype=torch.int32)
    labels = torch.tensor([sample["rating"] for sample in batch], dtype=torch.float)
    return user_ids, movie_ids, labels

Finally, we configure our training and validation datapipes to apply `_transform` to each batch of data using `map`.

In [8]:
preproc_train_dp = batched_train_dp.map(_transform)
preproc_val_dp = batched_val_dp.map(_transform)

At this point, `preproc_train_dp` and `preproc_val_dp` are set up to produce the data that our model expects.

## 2. Defining model

Next, we define the model we're going to train. We'll go with a simplified two-tower model `TwoTowerModel` resembling a matrix factorization model that attempts to learn a low-rank approximation of the user-movie ratings matrix. More specifically, we want to find matrices $U \in \mathbb{R}^{u \times d}$ and $M \in \mathbb{R}^{m \times d}$ such that $U M^T \approx A$, where each row in $U$ represents a user embedding of dimension $d$ and each row in $M$ a movie embedding also of dimension $d$. Once we find matrices $U$ and $M$, we can infer the rating that the $i$-th user gives the $j$-th movie as $u_i^T \cdot m_j^T$, i.e. the dot product of the $i$-th row in $U$ and $j$-th row in $M$.

`TwoTowerModel` represents $U$ and $M$ as embedding tables — instances of `torch.nn.Embedding`.

In [9]:
class TwoTowerModel(torch.nn.Module):
    def __init__(self, num_embeddings_0, num_embeddings_1, embedding_dim):
        super().__init__()
        self.model_0 = torch.nn.Embedding(num_embeddings_0, embedding_dim)
        self.model_1 = torch.nn.Embedding(num_embeddings_1, embedding_dim)
    
    def forward(self, input):
        embeddings_0 = self.model_0(input[0])
        embeddings_1 = self.model_1(input[1])
        return torch.sum(embeddings_0 * embeddings_1, axis=1)

## 3. Training and evaluating model
We're ready to train our model. Let's instantiate the model we just defined

In [10]:
model = TwoTowerModel(
    len(contig_user_ids),
    len(contig_movie_ids),
    32
)

, instantiate our loss function and optimizer

In [11]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-6)

, and define our train and test loops.

In [12]:
def train_loop(dp, model, loss_fn, optimizer):
    for batch, (users, movies, labels) in enumerate(dp):
        pred = model((users, movies))
        loss = loss_fn(pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(labels)
            print(f"loss: {loss:>7f}; batch: {batch}")

def test_loop(dp, model, loss_fn):
    test_loss = 0
    batch_count = 0
    with torch.no_grad():
        for batch, (users, movies, labels) in enumerate(dp):
            pred = model((users, movies))
            test_loss += loss_fn(pred, labels).item()
            batch_count += 1
    
    print(f"Test loss: {test_loss / batch_count}")

And now, we train.

In [13]:
epochs = 3

for __ in range(epochs):
    train_loop(preproc_train_dp, model, loss_fn, optimizer)
    test_loop(preproc_val_dp, model, loss_fn)

loss: 46.037308; batch: 0
loss: 38.465691; batch: 100
loss: 36.324226; batch: 200
loss: 32.601963; batch: 300
loss: 31.454372; batch: 400
loss: 28.976149; batch: 500
loss: 28.139290; batch: 600
loss: 26.033499; batch: 700
loss: 24.564535; batch: 800
loss: 24.771233; batch: 900
loss: 23.982235; batch: 1000
loss: 23.603422; batch: 1100
loss: 22.836342; batch: 1200
loss: 22.366283; batch: 1300
loss: 21.981915; batch: 1400
loss: 20.872860; batch: 1500
loss: 21.898285; batch: 1600
loss: 21.096714; batch: 1700
loss: 21.146002; batch: 1800
loss: 20.964083; batch: 1900
loss: 20.462320; batch: 2000
loss: 19.815897; batch: 2100
loss: 20.486073; batch: 2200
loss: 19.710440; batch: 2300
loss: 19.632444; batch: 2400
loss: 20.404284; batch: 2500
loss: 19.290865; batch: 2600
loss: 19.100807; batch: 2700
Test loss: 18.480480686511868
loss: 18.086700; batch: 0
loss: 17.338081; batch: 100
loss: 17.585329; batch: 200
loss: 17.180315; batch: 300
loss: 17.549704; batch: 400
loss: 17.645161; batch: 500
loss

We've got a trained model!

## 4. Finding similar movies
For kicks, let's see if we can use our model's trained embeddings to find movies that are most similar to some query movie. In theory, movies with embeddings that are similar should themselves be similar.

In [14]:
contig_to_movie_id = {v: k for k, v in contig_movie_ids.items()}

def get_topk_sim_movies(movie_id, k=20):
    embedding = model.model_1(torch.tensor([contig_movie_ids[movie_id]]))
    movie_embeddings = model.get_parameter("model_1.weight")
    movie_similarities = torch.sum(embedding * movie_embeddings, axis=1) / torch.maximum(torch.norm(embedding) * torch.norm(movie_embeddings, dim=1), torch.ones(movie_embeddings.shape[0]) * 1e-12)
    topk_sim = torch.topk(movie_similarities, 20)
    contig_ids = topk_sim.indices.tolist()
    return [
        (*movie_id_to_title_genre[contig_to_movie_id[movie_id]], contig_to_movie_id[movie_id]) 
        for movie_id in contig_ids
    ]

In [17]:
# Drive
get_topk_sim_movies(88129)

[('Drive (2011)', 'Crime|Drama|Film-Noir|Thriller', 88129),
 ('Wolf of Wall Street, The (2013)', 'Comedy|Crime|Drama', 106782),
 ('Ex Machina (2015)', 'Drama|Sci-Fi|Thriller', 115713),
 ('Shutter Island (2010)', 'Drama|Mystery|Thriller', 74458),
 ('Girl with the Dragon Tattoo, The (2011)', 'Drama|Thriller', 91658),
 ('True Grit (2010)', 'Western', 82459),
 ('Gone Girl (2014)', 'Drama|Thriller', 112556),
 ('Looper (2012)', 'Action|Crime|Sci-Fi', 96610),
 ('127 Hours (2010)', 'Adventure|Drama|Thriller', 81562),
 ('No Country for Old Men (2007)', 'Crime|Drama', 55820),
 ('Black Swan (2010)', 'Drama|Thriller', 81591),
 ('Big Short, The (2015)', 'Drama', 148626),
 ('Grand Budapest Hotel, The (2014)', 'Comedy|Drama', 109374),
 ('Skyfall (2012)', 'Action|Adventure|Thriller|IMAX', 96079),
 ('Dark Knight Rises, The (2012)', 'Action|Adventure|Crime|IMAX', 91529),
 ('Sherlock Holmes: A Game of Shadows (2011)',
  'Action|Adventure|Comedy|Crime|Mystery|Thriller',
  91542),
 ('Gran Torino (2008)', '

In [18]:
# Lost in Translation
get_topk_sim_movies(6711)

[('Lost in Translation (2003)', 'Comedy|Drama|Romance', 6711),
 ('Blade (1998)', 'Action|Horror|Thriller', 2167),
 ('O Brother, Where Art Thou? (2000)', 'Adventure|Comedy|Crime', 4027),
 ('Minority Report (2002)', 'Action|Crime|Mystery|Sci-Fi|Thriller', 5445),
 ('Bowling for Columbine (2002)', 'Documentary', 5669),
 ('Lord of War (2005)', 'Action|Crime|Drama|Thriller|War', 36529),
 ('Dark City (1998)', 'Adventure|Film-Noir|Sci-Fi|Thriller', 1748),
 ('Rocky (1976)', 'Drama', 1954),
 ('Best in Show (2000)', 'Comedy', 3911),
 ('Deliverance (1972)', 'Adventure|Drama|Thriller', 2871),
 ('Eternal Sunshine of the Spotless Mind (2004)',
  'Drama|Romance|Sci-Fi',
  7361),
 ('Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)',
  'Action|Drama|Romance',
  3996),
 ('Fahrenheit 9/11 (2004)', 'Documentary', 8622),
 ('Back to the Future Part III (1990)',
  'Adventure|Comedy|Sci-Fi|Western',
  2012),
 ('Boogie Nights (1997)', 'Drama', 1673),
 ('Run Lola Run (Lola rennt) (1998)', 'Action|Crime', 

In [21]:
# Ratatouille
get_topk_sim_movies(50872)

[('Ratatouille (2007)', 'Animation|Children|Drama', 50872),
 ('Harry Potter and the Goblet of Fire (2005)',
  'Adventure|Fantasy|Thriller|IMAX',
  40815),
 ('Toy Story 2 (1999)', 'Adventure|Animation|Children|Comedy|Fantasy', 3114),
 ('Ice Age (2002)', 'Adventure|Animation|Children|Comedy', 5218),
 ('Monsters, Inc. (2001)',
  'Adventure|Animation|Children|Comedy|Fantasy',
  4886),
 ('Harry Potter and the Chamber of Secrets (2002)', 'Adventure|Fantasy', 5816),
 ('Wedding Singer, The (1998)', 'Comedy|Romance', 1777),
 ('Armageddon (1998)', 'Action|Romance|Sci-Fi|Thriller', 1917),
 ('WALL·E (2008)', 'Adventure|Animation|Children|Romance|Sci-Fi', 60069),
 ('Finding Nemo (2003)', 'Adventure|Animation|Children|Comedy', 6377),
 ('Indiana Jones and the Temple of Doom (1984)',
  'Action|Adventure|Fantasy',
  2115),
 ('Signs (2002)', 'Horror|Sci-Fi|Thriller', 5502),
 ('Toy Story 3 (2010)',
  'Adventure|Animation|Children|Comedy|Fantasy|IMAX',
  78499),
 ('My Big Fat Greek Wedding (2002)', 'Comed

What do you think? Can we do better?